## Daten zusammenfügen und erster Blick auf die Daten

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Datensatz einlesen
umsatzdaten = pd.read_csv('0_DataPreparation/umsatzdaten_gekuerzt.csv')
wetter = pd.read_csv('0_DataPreparation/wetter.csv')
kiwo = pd.read_csv('0_DataPreparation/kiwo.csv')

# Daten zusammenführen als outer join
merged_data = pd.merge(umsatzdaten, wetter, on='Datum', how='outer')
merged_data = pd.merge(merged_data, kiwo, on='Datum', how='outer')

# Umsatzdaten anzeigen
print(merged_data.head())
print(merged_data.info())

# Deskriptive Statistik für Umsatzdaten
print(merged_data.describe())
print(merged_data.isnull())



        Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0  2012-01-01          NaN     NaN         8.0      9.8250   
1  2012-01-02          NaN     NaN         7.0      7.4375   
2  2012-01-03          NaN     NaN         8.0      5.5375   
3  2012-01-04          NaN     NaN         4.0      5.6875   
4  2012-01-05          NaN     NaN         6.0      5.3000   

   Windgeschwindigkeit  Wettercode  KielerWoche  
0                 14.0        58.0          NaN  
1                 12.0         NaN          NaN  
2                 18.0        63.0          NaN  
3                 19.0        80.0          NaN  
4                 23.0        80.0          NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10119 entries, 0 to 10118
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Datum                10119 non-null  object 
 1   Warengruppe          9334 non-null   float64
 2   Umsatz        

## Temperatur in kategoriale Variable transformieren und Feiertage hinzufügen

In [22]:
# Temperatur in kategoriale Variable umwandeln (Kategorien numerisch kodiert)
merged_data['Temperatur_kat'] = pd.cut(merged_data['Temperatur'], bins=[-10, 0, 20, 45], labels=[0, 1, 2])


In [23]:
import pandas as pd
import holidays

# Liste der Feiertage für Deutschland
de_holidays = holidays.Germany()

# Datumsbereich
start_date = '2012-01-01'
end_date = '2019-08-01'

# Liste aller Tage im Datumsbereich
date_range = pd.date_range(start=start_date, end=end_date)

# Liste zur Speicherung des Feiertagsstatus
holiday_status = []

# Überprüfung jedes Datums auf Feiertag
for date in date_range:
    if date in de_holidays:
        holiday_status.append(1)  # Feiertag
    else:
        holiday_status.append(0)  # normaler Tag

# Erstellen eines DataFrame mit Datum und Feiertagsstatus
holiday_df = pd.DataFrame({'Datum': date_range, 'Feiertag': holiday_status})

# Anzeigen der ersten Zeilen des DataFrame
print(holiday_df.head())

# Datum in umsatzdaten in datetime umwandeln
merged_data['Datum'] = pd.to_datetime(merged_data['Datum'])

# holiday_df mit umsatzdaten zusammenführen
merged_data = pd.merge(merged_data, holiday_df, on='Datum', how='outer')

# Anzeigen der ersten Zeilen des DataFrame
print(merged_data)

       Datum  Feiertag
0 2012-01-01         1
1 2012-01-02         0
2 2012-01-03         0
3 2012-01-04         0
4 2012-01-05         0
           Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0     2012-01-01          NaN     NaN         8.0      9.8250   
1     2012-01-02          NaN     NaN         7.0      7.4375   
2     2012-01-03          NaN     NaN         8.0      5.5375   
3     2012-01-04          NaN     NaN         4.0      5.6875   
4     2012-01-05          NaN     NaN         6.0      5.3000   
...          ...          ...     ...         ...         ...   
10280 2019-07-28          NaN     NaN         3.0     23.3500   
10281 2019-07-29          NaN     NaN         6.0     25.2500   
10282 2019-07-30          NaN     NaN         7.0     20.7375   
10283 2019-07-31          NaN     NaN         6.0     20.4500   
10284 2019-08-01          NaN     NaN         5.0     21.0625   

       Windgeschwindigkeit  Wettercode  KielerWoche Temperatur_kat  Feiertag  
0 

## Variable Wochenende hinzufügen

In [24]:
merged_data['Datum'] = pd.to_datetime(merged_data['Datum'])
merged_data['Wochenende'] = merged_data['Datum'].dt.dayofweek >= 5


## Variable Temperaturabweichung vom Monatsmittel hinzufügen und binäre Variablen numerisch kodieren

In [25]:
# Variable für Temperaturabweichung um Mittelwert der Temperatur für jeden Monat berechnen
# Um die Monate zu extrahieren
merged_data['Monat'] = merged_data['Datum'].dt.month

# Mittelwerte der Temperatur für jeden Monat berechnen
monatsmittel_temp = merged_data.groupby('Monat')['Temperatur'].transform('mean')

# Um die Temperaturabweichung zu berechnen
merged_data['Temp_abw'] = (merged_data['Temperatur'] - monatsmittel_temp).abs() > 5

# Kieler Woche, Wochenende und Temp.abw binär mit 0 und 1 kodieren
merged_data['KielerWoche'] = merged_data['KielerWoche'].fillna(0).astype(int)
merged_data['Wochenende'] = merged_data['Wochenende'].fillna(0).astype(int)
merged_data['Temp_abw'] = merged_data['Temp_abw'].fillna(0).astype(int)

# Anzeigen der ersten Zeilen des DataFrame mit der neuen Spalte
print(merged_data.head())

# Wie oft True für Temp_abw
print(merged_data['Temp_abw'].value_counts())


       Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0 2012-01-01          NaN     NaN         8.0      9.8250   
1 2012-01-02          NaN     NaN         7.0      7.4375   
2 2012-01-03          NaN     NaN         8.0      5.5375   
3 2012-01-04          NaN     NaN         4.0      5.6875   
4 2012-01-05          NaN     NaN         6.0      5.3000   

   Windgeschwindigkeit  Wettercode  KielerWoche Temperatur_kat  Feiertag  \
0                 14.0        58.0            0              1         1   
1                 12.0         NaN            0              1         0   
2                 18.0        63.0            0              1         0   
3                 19.0        80.0            0              1         0   
4                 23.0        80.0            0              1         0   

   Wochenende  Monat  Temp_abw  
0           1      1         1  
1           0      1         0  
2           0      1         0  
3           0      1         0  
4          

## Bewölkung und Windgeschwindigkeit in kategoriale Variable transformieren

In [26]:
# Variable erstellen, die die Bewölkung in kategoriale Variable umwandelt
merged_data['Bewoelkung_kat'] = pd.cut(merged_data['Bewoelkung'], bins=[0, 3, 6, 9], labels=[0, 1, 2])

# Anzeigen der ersten Zeilen des DataFrame mit der neuen Spalte
print(merged_data.head())

# Nur minimal größerer Zusammenhang zwischen Bewölkung und Umsatz (siehe Korrelation)

# Windgeschwindigkeit in kategoriale Variable (minimum 3 und mximum 35) umwandeln
merged_data['Windgeschwindigkeit_kat'] = pd.cut(merged_data['Windgeschwindigkeit'], bins=[0, 5, 10, 15, 20, 25, 30, 35], labels=[0, 1, 2, 3, 4, 5, 6])

# Anzeigen der ersten Zeilen des DataFrame mit der neuen Spalte
print(merged_data.head())
print(merged_data.isna().sum())

       Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0 2012-01-01          NaN     NaN         8.0      9.8250   
1 2012-01-02          NaN     NaN         7.0      7.4375   
2 2012-01-03          NaN     NaN         8.0      5.5375   
3 2012-01-04          NaN     NaN         4.0      5.6875   
4 2012-01-05          NaN     NaN         6.0      5.3000   

   Windgeschwindigkeit  Wettercode  KielerWoche Temperatur_kat  Feiertag  \
0                 14.0        58.0            0              1         1   
1                 12.0         NaN            0              1         0   
2                 18.0        63.0            0              1         0   
3                 19.0        80.0            0              1         0   
4                 23.0        80.0            0              1         0   

   Wochenende  Monat  Temp_abw Bewoelkung_kat  
0           1      1         1              2  
1           0      1         0              2  
2           0      1         0  

## Zeitvariablen und Lagged Variables hinzufügen

In [27]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

# Neue Zeitvariablen hinzufügen
merged_data['Wochentag'] = pd.to_datetime(merged_data['Datum']).dt.dayofweek
merged_data['Kalenderwoche'] = pd.to_datetime(merged_data['Datum']).dt.isocalendar().week
merged_data['Jahreszeit'] = pd.to_datetime(merged_data['Datum']).dt.month % 12 // 3 + 1

# Lagged Variables hinzufügen (Umsatz einen Tag und eine Woche zuvor)
merged_data['Umsatz_lag1'] = merged_data['Umsatz'].shift(1)
merged_data['Umsatz_lag7'] = merged_data['Umsatz'].shift(7)

# Feiertag in den nächsten drei Tagen
merged_data['Feiertag_in_3_tagen'] = merged_data['Feiertag'].rolling(window=4, min_periods=1).max().shift(-3)

# Durchschnittlicher Umsatz der letzten drei Tage vor einem Feiertag
# Initialisieren der neuen Spalte mit NaN
merged_data['durchschnittlicher_Umsatz_vor_Feiertag'] = np.nan

# Berechnung des durchschnittlichen Umsatzes der letzten drei Tage vor einem Feiertag
for i in range(len(merged_data)):
    if merged_data.loc[i, 'Feiertag_in_3_tagen'] == 1:
        merged_data.loc[i, 'durchschnittlicher_Umsatz_vor_Feiertag'] = merged_data.loc[i-3:i-1, 'Umsatz'].mean()

# Anzeigen von random Zeilen des DataFrame mit den neuen Spalten
print(merged_data.sample(5))
print(merged_data)

# Zeilen anzeigen mit Feiertag_in_3_tagen
print(merged_data[merged_data['Feiertag_in_3_tagen'] == 1])

# Anzeigen welche Daten als Feiertage markiert sind
print(merged_data[merged_data['Feiertag'] == 1])


          Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
5304 2016-01-25          1.0  101.660735         8.0      9.0000   
4674 2015-09-28          5.0  248.782546         2.0     14.6250   
4791 2015-10-22          2.0  409.101521         7.0     12.2500   
512  2013-05-27          NaN         NaN         6.0     14.7875   
6405 2016-09-06          5.0  257.278300         2.0     21.7500   

      Windgeschwindigkeit  Wettercode  KielerWoche Temperatur_kat  Feiertag  \
5304                  9.0        45.0            0              1         0   
4674                  7.0         0.0            0              1         0   
4791                 10.0        61.0            0              1         0   
512                  10.0         NaN            0              1         0   
6405                  6.0         NaN            0              2         0   

      ...  Temp_abw  Bewoelkung_kat  Windgeschwindigkeit_kat Wochentag  \
5304  ...         1               2       

## Korrelationen

In [28]:
# Korrelationsmatrix für Umsatzdaten
print(merged_data.corr())


                                           Datum  Warengruppe    Umsatz  \
Datum                                   1.000000    -0.006639 -0.086901   
Warengruppe                            -0.006639     1.000000 -0.062589   
Umsatz                                 -0.086901    -0.062589  1.000000   
Bewoelkung                             -0.019253     0.023795 -0.089916   
Temperatur                             -0.019917    -0.047015  0.222935   
Windgeschwindigkeit                    -0.020052    -0.009078  0.014705   
Wettercode                             -0.114740     0.009547 -0.010091   
KielerWoche                             0.033931    -0.010207  0.058244   
Temperatur_kat                          0.027973    -0.022894  0.150435   
Feiertag                                0.000659    -0.006163  0.043574   
Wochenende                              0.002347     0.000457  0.163097   
Monat                                  -0.044090     0.080817  0.013889   
Temp_abw                 

## Lineares Modell ohne Interaktionseffekte

In [50]:
# Fit the linear model (erster Versuch)
mod = smf.ols('Umsatz ~ C(Warengruppe)  + Temperatur + C(Feiertag) + C(Jahreszeit) + C(Wochenende) + Umsatz_lag1', data=merged_data).fit()

# Print the summary
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.732
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     2106.
Date:                Tue, 11 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:11:50   Log-Likelihood:                -53273.
No. Observations:                9287   AIC:                         1.066e+05
Df Residuals:                    9274   BIC:                         1.067e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                43.47

## Lineares Modell mit Interaktionseffekten

In [45]:
# Modell mit Interaktionseffekten
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
import itertools

# Formel für die multiple lineare Regression mit ausgewählten Interaktionseffekten
mod_int = smf.ols('Umsatz ~ C(Warengruppe) + Temperatur + C(Feiertag) + C(Jahreszeit) + C(Wochenende) + Umsatz_lag1 + Umsatz_lag7 + C(KielerWoche)*C(Warengruppe) + Wettercode*Temperatur + Monat*Temperatur', data = merged_data).fit()

# Print the summary
print(mod_int.summary())




                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.726
Model:                            OLS   Adj. R-squared:                  0.725
Method:                 Least Squares   F-statistic:                     835.5
Date:                Tue, 11 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:00:06   Log-Likelihood:                -39774.
No. Observations:                6959   AIC:                         7.959e+04
Df Residuals:                    6936   BIC:                         7.975e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

## Daten aufteilen und Vorhersagen testen

In [55]:
# Daten mit Zeitreihenstruktur aufteilen
# Ensure the data is sorted by date
data = merged_data.sort_values(by='Datum')
print (merged_data.head())
# Define your date thresholds
train_end_date = '2017-07-31'
validation_end_date = '2018-07-31'
# Split the data based on the date thresholds
train_data = data[data['Datum'] <= train_end_date]
validation_data = data[(data['Datum'] > train_end_date) & (data['Datum'] <= validation_end_date)]
test_data = data[data['Datum'] > validation_end_date]
# Check the dimensions of the datasets
print("Training dataset dimensions:", train_data.shape)
print("Validation dataset dimensions:", validation_data.shape)
print("Test dataset dimensions:", test_data.shape)


       Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0 2012-01-01          NaN     NaN         8.0      9.8250   
1 2012-01-02          NaN     NaN         7.0      7.4375   
2 2012-01-03          NaN     NaN         8.0      5.5375   
3 2012-01-04          NaN     NaN         4.0      5.6875   
4 2012-01-05          NaN     NaN         6.0      5.3000   

   Windgeschwindigkeit  Wettercode  KielerWoche Temperatur_kat  Feiertag  ...  \
0                 14.0        58.0            0              1         1  ...   
1                 12.0         NaN            0              1         0  ...   
2                 18.0        63.0            0              1         0  ...   
3                 19.0        80.0            0              1         0  ...   
4                 23.0        80.0            0              1         0  ...   

   Temp_abw  Bewoelkung_kat  Windgeschwindigkeit_kat Wochentag Kalenderwoche  \
0         1               2                        2         6    

In [52]:
# Create a new house with the following features
new_day = pd.DataFrame({
'Temperatur': [20], 
'Warengruppe': [3],
"Feiertag" : [1],
"Jahreszeit" : [2],
"Wochenende" : [1],
"Umsatz_lag1" : [100.80162265371] 
})
# Use the model to predict the sales of the new day
predicted_sales = mod.predict(new_day)
print(f"The predicted sales for the new day is: {predicted_sales[0]}")


The predicted sales for the new day is: 224.18552938937597
